<a href="https://colab.research.google.com/github/cristianmejia00/clustering/blob/main/Topic_Models_using_BERTopic_TOPIC_MODEL_20241101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling with BERTopic

🔴 copied from the [Kubota Colab](https://colab.research.google.com/drive/1YsDp5_qGXGJKsEXsS8DO8CA_lqZc6EpA).  

`Topic Models` are methods to automatically organize a corpus of text into topics.

Topic Model process:
1. Data preparation
2. Tranform text to numeric vectors
3. Multidimensionality reduction
4. Clustering
5. Topic analysis
6. Cluster assignation


This notebook uses the library `BERTopic` which is a one-stop solution for topic modeling including handy functions for plotting and analysis. However, BERTopic does not have a function to extract the X and Y coords from UMAP. If we need the coordinates then use the notebooks `Topic_Models_using_Transformers` instead. In any other situation, when a quick analysis is needed this notebook may be better.

This notebook is also the one needed for the heatmap codes included in this folder.

`BERTopic` is Python library that handles steps 2 to 6.
BERT topic models use the transformer architechture to generate the embeds (i.e. the vector or numeric representation of words) and are currently the state-of-the-art method for vectorization.

This notebook shows how to use it.

---
Reading:
[Topic Modeling with Deep Learning Using Python BERTopic](https://medium.com/grabngoinfo/topic-modeling-with-deep-learning-using-python-bertopic-cf91f5676504)
[Advanced Topic Modeling with BERTopic](https://www.pinecone.io/learn/bertopic/)


# Requirements

## Packages installation and initialization

In [ ]:
#!pip install bertopic[visualization]

In [21]:
import pandas as pd
import numpy as np
import time
import math
import uuid
import re
import os
import json
import pickle
from datetime import date
from itertools import compress
from bertopic import BERTopic
from umap import UMAP
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer

## Connect your Google Drive

In [22]:
# from google.colab import drive
# drive.mount('/content/drive')

In [23]:
def find_e_keys(dictionary):
    # List comprehension to find keys starting with 'e'
    e_keys = [key for key in dictionary if str(key).lower().startswith('e')]
    return e_keys

# 🔴 Input files and options

Go to your Google Drive and create a folder in the root directory. We are going to save all related data in that directory.
Upload the dataset of news into the above folder.
- The dataset should be a `.csv` file.
- Every row in the dataset is a document
- It can any kind of columns. Some columns must contain the text we want to analyze. For example, a dataset of academic articles may contain a "Title" and/or "Abstract" column.

In [24]:
# The bibliometrics folder
# Colab
ROOT_FOLDER_PATH = "drive/MyDrive/Bibliometrics_Drive"

# Mac
ROOT_FOLDER_PATH = "/Users/cristian/Library/CloudStorage/GoogleDrive-cristianmejia00@gmail.com/My Drive/Bibliometrics_Drive"

# Change to the name of the folder where the dataset is uploaded inside the above folder
project_folder = 'Q331_TI_sdgs_TS_policy'

analysis_id = 'a01_tm__f01_e01__km01'

# Filtered label
settings_directive = "settings_analysis_directive_2025-05-10-12-53.json"

In [25]:
# Read settings
with open(f'{ROOT_FOLDER_PATH}/{project_folder}/{analysis_id}/{settings_directive}', 'r') as file:
    settings = json.load(file)

In [26]:
# Input dataset
dataset_file_path = f"{ROOT_FOLDER_PATH}/{settings['metadata']['project_folder']}/{settings['metadata']['filtered_folder']}/dataset_raw_cleaned.csv"

In [27]:
# Function to save files
def save_as_csv(df, save_name_without_extension, with_index):
    "usage: `save_as_csv(dataframe, 'filename')`"
    df.to_csv(f"{ROOT_FOLDER_PATH}/{save_name_without_extension}.csv", index=with_index)
    print("===\nSaved: ", f"{ROOT_FOLDER_PATH}/{save_name_without_extension}.csv")

In [28]:
# prompt: a function to save object to a pickle file
def save_object_as_pickle(obj, filename):
  """
  Saves an object as a pickle file.

  Args:
      obj: The object to be saved.
      filename: The filename of the pickle file.
  """
  with open(filename, "wb") as f:
    pickle.dump(obj, f)


In [29]:
# prompt: a function to load pickle object given a path
def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)


In [30]:
# Open the data file
df = pd.read_csv(f"{dataset_file_path}", encoding='latin-1')
print(df.shape)
df.head()

(3031, 42)


,X_N,uuid,PT,AU,AF,TI,SO,LA,DT,DE,...,AR,DI,PG,WC,SC,OA,UT,Countries,IsoCountries,Institutions
0,1,8c7848eb-4e3a-4f87-98f2-4c1638c4d195,J,"Fasoulis, I","Fasoulis, Ioannis",Governing the oceans: A study into Norway's oc...,REGIONAL STUDIES IN MARINE SCIENCE,English,Article,Sustainable Development Goals; Ocean governanc...,...,101983,10.1016/j.rsma.2021.101983,15,Ecology; Marine & Freshwater Biology,Environmental Sciences & Ecology; Marine & Fre...,Green Accepted,WOS:000704445400012,united kingdom,GBR,univ strathclyde
1,2,46d829bb-c921-4040-b714-592e62dfeac0,J,"Rozas-VÃ¡squez, D; Spyra, M; Jorquera, F; Moli...","Rozas-Vasquez, Daniel; Spyra, Marcin; Jorquera...",Ecosystem Services Supply from Peri-Urban Land...,LAND,English,Review,peri-urban landscape; ecosystem services; sust...,...,2006,10.3390/land11112006,18,Environmental Studies,Environmental Sciences & Ecology,gold,WOS:000895505400001,chile; germany; poland,CHL; DEU; POL,univ catolica temuco; martin luther univ halle...
2,3,c0a1ad65-9abf-4c4a-b104-064855237ee4,J,"Gutberlet, J","Gutberlet, Jutta",Grassroots waste picker organizations addressi...,WORLD DEVELOPMENT,English,Article,Waste pickers; Social and Solidarity Economy; ...,...,105195,10.1016/j.worlddev.2020.105195,12,Development Studies; Economics,Development Studies; Business & Economics,Green Published,WOS:000601162800011,canada,CAN,univ victoria
3,4,44e80506-efaf-4bd6-a26f-b5f33589e950,J,"Opoku, A; Ahmed, V; Ofori, G","Opoku, Alex; Ahmed, Vian; Ofori, George",Realising the sustainable development goals th...,RESOURCES CONSERVATION AND RECYCLING,English,Editorial Material,Construction; Organisational learning; Circula...,...,106427,10.1016/j.resconrec.2022.106427,6,"Engineering, Environmental; Environmental Scie...",Engineering; Environmental Sciences & Ecology,NaN,WOS:000809762800006,u arab emirates; united kingdom,ARE; GBR,univ sharjah; amer univ sharjah; london south ...
4,5,2f1fe54b-c88f-44bb-8209-24b5c334ae3c,J,"Firoiu, D; Ionescu, GH; Cismas, CM; Costin, MP...","Firoiu, Daniela; Ionescu, George H.; Cismas, C...",Sustainable Production and Consumption in EU M...,SUSTAINABILITY,English,Article,sustainable development; 2030 agenda; SDG 12; ...,...,1537,10.3390/su17041537,26,Green & Sustainable Science & Technology; Envi...,Science & Technology - Other Topics; Environme...,gold,WOS:001431741100001,romania,ROU,romanian amer univ; west univ timisoara; univ ...




---



## PART 2: Topic Model

In [31]:
# bibliometrics_folder
# project_folder
# project_name_suffix
# ROOT_FOLDER_PATH = f"drive/MyDrive/{bibliometrics_folder}"

#############################################################
# Embeddings folder
embeddings_folder_name = settings['tmo']['embeds_folder']

# Which column has the year of the documents?
my_year = settings['tmo']['year_column']

# Number of topics. Select the number of topics to extract.
# Choose 0, for automatic detection.
n_topics = settings['tmo']['n_topics']

# Minimum number of documents per topic
min_topic_size = settings['tmo']['min_topic_size']

In [32]:
# Get the embeddings back.
embeddings = load_pickle(f"{ROOT_FOLDER_PATH}/{project_folder}/{settings['metadata']['filtered_folder']}/{embeddings_folder_name}/embeddings.pck")
corpus =     pd.read_csv(f"{ROOT_FOLDER_PATH}/{project_folder}/{settings['metadata']['filtered_folder']}/{embeddings_folder_name}/corpus.csv").reset_index(drop=True)

In [33]:
# Combine embeddings
documents = corpus.text.to_list()

In [34]:
# corpus['uuid'] = [uuid.uuid4() for _ in range(len(corpus.index))]
# corpus['X_N'] = [i for i in range(1, len(corpus.index)+1)]

In [35]:
len(documents)

3031

In [36]:
#len(embeddings) == len(documents)
len(embeddings['embeddings']) == len(documents)

True

In [37]:
from hdbscan.hdbscan_ import HDBSCAN
# Execute the topic model.
# I suggest changing the values marked with #<---
# The others are the default values and they'll work fine in most cases.
# This will take several minutes to finish.

# Initiate UMAP
umap_model = UMAP(n_neighbors=15,
                  n_components=5,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)

if n_topics == 0:
  # Initiate topic model with HDBScan (Automatic topic selection)
  topic_model_params = HDBSCAN(min_cluster_size=min_topic_size,
                               metric='euclidean',
                               cluster_selection_method='eom',
                               prediction_data=True)
else:
  # Initiate topic model with K-means (Manual topic selection)
  topic_model_params = KMeans(n_clusters = n_topics)

# Initiate BERTopic
topic_model = BERTopic(umap_model = umap_model,
                       hdbscan_model = topic_model_params,
                       min_topic_size=min_topic_size,
                       #nr_topics=15,          #<--- Footnote 1
                       n_gram_range=(1,3),
                       language='english',
                       calculate_probabilities=True,
                       verbose=True)



# Footnote 1: This controls the number of topics we want AFTER clustering.
# Add a hashtag at the beggining to use the number of topics returned by the topic model.
# When using HDBScan nr_topics will be obtained after orphans removal, and there is no warranty that `nr_topics < HDBScan topics`.
# thus, with HDBScan `nr_topics` means N topics OR LESS.
# When using KMeans nr_topics can be used to further reduce the number of topics.
# We use the topics as returned by the topic model. So we do not need to activate it here.

In [38]:
# Compute topic model
#topics, probabilities = topic_model.fit_transform(documents, embeddings)
topics, probabilities = topic_model.fit_transform(documents, embeddings['embeddings'])

2025-05-10 13:11:47,795 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-10 13:11:55,861 - BERTopic - Dimensionality - Completed ✓
2025-05-10 13:11:55,862 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-10 13:11:55,887 - BERTopic - Cluster - Completed ✓
2025-05-10 13:11:55,890 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-05-10 13:11:57,783 - BERTopic - Representation - Completed ✓


In [39]:
# Get the list of topics
# Topic = the topic number. From the largest topic.
#         "-1" is the generic topic. Genericr keywords are aggegrated here.
# Count = Documents assigned to this topic
# Name = Top 4 words of the topic based on probability
# Representation = The list of words representing this topic
# Representative_Docs = Documents assigned to this topic
tm_summary = topic_model.get_topic_info()
tm_summary

,Topic,Count,Name,Representation,Representative_Docs
0,0,118,0_energy_the_and_of,"[energy, the, and, of, in, to, renewable, sust...",[Mapping Potential for Improving Rural Energy ...
1,1,107,1_the_of_governance_and,"[the, of, governance, and, development, to, in...",[From Millennium to Sustainable Development Go...
2,2,103,2_the_of_education_and,"[the, of, education, and, in, universities, hi...",[The implementation of the SDGs in universitie...
3,3,103,3_health_and_the_to,"[health, and, the, to, of, in, for, global, he...",[A stakeholder group assessment of interaction...
4,4,96,4_energy_green_environmental_renewable,"[energy, green, environmental, renewable, the,...",[Role of renewable and nonrenewable energy con...
...,...,...,...,...,...
57,57,14,57_disaster_and_risk_the,"[disaster, and, risk, the, events, sfdrr, drr,...",[Addressing the interplay of the Sendai Framew...
58,58,12,58_heritage_cultural_cultural heritage_the,"[heritage, cultural, cultural heritage, the, a...",[Globalization Cultural Heritage Management an...
59,59,12,59_nurses_nursing_goals_the,"[nurses, nursing, goals, the, to, and, sdgs, t...",[Nursing and the Sustainable Development Goals...
60,60,12,60_disaster risk_disaster_risk_disaster risk r...,"[disaster risk, disaster, risk, disaster risk ...",[Number of Number of countries that adopt and ...


In [40]:
# Save the topic model assets
tm_folder_path = f'{ROOT_FOLDER_PATH}/{project_folder}/{settings["metadata"]["analysis_id"]}'

if not os.path.exists(tm_folder_path):
  !mkdir $tm_folder_path

tm_summary.to_csv(f'{tm_folder_path}/topic_model_info.csv', index=False)

In [41]:
# Number of topics found
found_topics = max(tm_summary.Topic) + 1
found_topics

62

In [42]:
# Confirm all documents are assigned
sum(tm_summary.Count) == len(corpus)

True

In [43]:
# Get top 10 terms for a topic
topic_model.get_topic(0)

[('energy', 0.029997663038704648),
 ('the', 0.01165301782033799),
 ('and', 0.010853522883957308),
 ('of', 0.010181684582470258),
 ('in', 0.009019562991501246),
 ('to', 0.009009062423404322),
 ('renewable', 0.007129674157699376),
 ('sustainable', 0.006859922266315907),
 ('for', 0.006493251391623176),
 ('access', 0.006438177327144445)]

In [44]:
# Get the top 10 documents for a topic
topic_model.get_representative_docs(0)

['Mapping Potential for Improving Rural Energy Services in Kyrgyzstan Factors for Achieving Sustainable Development Goals in the Community Context Energy is a pivotal element for overall development Therefore affordable and clean energy for all Sustainable Development Goal  is one of the important elements Despite a suitable approach Kyrgyzstan lags behind to achieve its Sustainable Development Goals by  Improving access to energy can significantly impact the progress of many other sustainable development targets However access to reliable sustainable and adequate energy is a crucial task in rural Kyrgyzstan Taking into consideration the energy situation and the importance of the availability of energy services the overarching aim of the presented article is to map the potential of improving rural energy services to foster overall sustainable development goals The article presents the most relevant information about rural energy usage behaviour based on three primary pillars energy dem

In [45]:
# Others

# # Get the number of documents per topic (same as in the table above)
# topic_model.get_topic_freq(0)

# # Get the main keywords per topic
# topic_model.get_topics()

In [46]:
# Print the parameters used. (For reporting)
topic_model.get_params()

{'calculate_probabilities': True,
 'ctfidf_model': ClassTfidfTransformer(),
 'embedding_model': None,
 'hdbscan_model': KMeans(n_clusters=62),
 'language': 'english',
 'low_memory': False,
 'min_topic_size': 5,
 'n_gram_range': (1, 3),
 'nr_topics': None,
 'representation_model': None,
 'seed_topic_list': None,
 'top_n_words': 10,
 'umap_model': UMAP(angular_rp_forest=True, metric='cosine', min_dist=0.0, n_components=5, n_jobs=1, random_state=100, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}),
 'vectorizer_model': CountVectorizer(ngram_range=(1, 3)),
 'verbose': True,
 'zeroshot_min_similarity': 0.7,
 'zeroshot_topic_list': None}

In [47]:
tm_params = dict(topic_model.get_params())
for key, value in tm_params.items():
    tm_params[key]=  str(value)
with open(f'{tm_folder_path}/topic_model_params.json', 'w') as f:
    json.dump(tm_params, f, ensure_ascii=False, indent=4)
    print('Done')

Done


In [48]:
# Get the topic score for each paper and its assigned topic
topic_distr, _ = topic_model.approximate_distribution(documents, batch_size=1000)
distributions = [distr[topic] if topic != -1 else 0 for topic, distr in zip(topics, topic_distr)]

100%|██████████| 4/4 [00:03<00:00,  1.06it/s]


In [49]:
topic_distr

array([[0.01757269, 0.05070063, 0.01944976, ..., 0.0274568 , 0.        ,
        0.01619874],
       [0.02103762, 0.03650038, 0.02365354, ..., 0.00951741, 0.        ,
        0.01467478],
       [0.0183611 , 0.04320568, 0.02718617, ..., 0.00759131, 0.        ,
        0.01782156],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [50]:
# Document information. Including the topic assignation
dataset_clustering_results = topic_model.get_document_info(documents, df = corpus, metadata={"Score": distributions})

# Standar format for report analysis
dataset_clustering_results = dataset_clustering_results.drop(columns=['text'])
dataset_clustering_results['X_E'] = dataset_clustering_results['Score']
dataset_clustering_results['X_C'] = dataset_clustering_results['Topic'] + 1
dataset_clustering_results['level0'] = dataset_clustering_results['Topic'] + 1
dataset_clustering_results['cl99'] = False
dataset_clustering_results['cl-99'] = False
dataset_clustering_results.head()

,UT,uuid,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Representative_document,Score,X_E,X_C,level0,cl99,cl-99
0,WOS:000704445400012,8c7848eb-4e3a-4f87-98f2-4c1638c4d195,Governing the oceans A study into Norways ocea...,15,15_marine_and_fisheries_the,"[marine, and, fisheries, the, ocean, of, to, s...",[Rethinking marine resource governance for the...,marine - and - fisheries - the - ocean - of - ...,False,0.063335,0.063335,16,16,False,False
1,WOS:000895505400001,46d829bb-c921-4040-b714-592e62dfeac0,Ecosystem Services Supply from PeriUrban Lands...,45,45_the_ecosystem_and_to,"[the, ecosystem, and, to, of, ecosystem servic...",[Distilling the role of ecosystem services in ...,the - ecosystem - and - to - of - ecosystem se...,False,0.039901,0.039901,46,46,False,False
2,WOS:000601162800011,c0a1ad65-9abf-4c4a-b104-064855237ee4,Grassroots waste picker organizations addressi...,30,30_waste_waste management_management_and,"[waste, waste management, management, and, the...",[Towards a sustainable future Nexus between th...,waste - waste management - management - and - ...,False,0.143070,0.143070,31,31,False,False
3,WOS:000809762800006,44e80506-efaf-4bd6-a26f-b5f33589e950,Realising the sustainable development goals th...,54,54_construction_the_construction industry_in,"[construction, the, construction industry, in,...",[Sustainable development goals under threat th...,construction - the - construction industry - i...,False,0.164792,0.164792,55,55,False,False
4,WOS:001431741100001,2f1fe54b-c88f-44bb-8209-24b5c334ae3c,Sustainable Production and Consumption in EU M...,43,43_the_consumption_and_of,"[the, consumption, and, of, ce, circular, to, ...",[How can the circular economy support the adva...,the - consumption - and - of - ce - circular -...,False,0.060824,0.060824,44,44,False,False


In [51]:
# Save the dataframe
dataset_clustering_results.to_csv(f'{tm_folder_path}/dataset_minimal.csv', index=False)

In [52]:
# Save the topic model
topic_model.save(f'{tm_folder_path}/topic_model_object.pck')

2025-05-10 13:12:03,087 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.




---

